In [ ]:
CREATE SOURCE IF NOT EXISTS client_source (
    datum_registracije TIMESTAMP,
    jedinstveni_identifikator INT,
    starost INT,
    pol VARCHAR,
    bracno_stanje VARCHAR
)
WITH (
   connector='kafka',
   topic='client_events',
   properties.bootstrap.server='redpanda-0:9092'
) FORMAT PLAIN ENCODE JSON;


In [ ]:
CREATE SOURCE IF NOT EXISTS product_change_source (
    jedinstveni_identifikator INT,
    datum TIMESTAMP,
    proizvod VARCHAR,
    indikator VARCHAR
)
WITH (
   connector='kafka',
   topic='product_change_events',
   properties.bootstrap.server='redpanda-0:9092'
) FORMAT PLAIN ENCODE JSON;

In [ ]:
CREATE SOURCE IF NOT EXISTS transaction_source (
    jedinstveni_identifikator INT,
    datum_transakcije TIMESTAMP,
    vrsta_transakcije VARCHAR,
    iznos_transakcije REAL
)
WITH (
   connector='kafka',
   topic='transaction_events',
   properties.bootstrap.server='redpanda-0:9092'
) FORMAT PLAIN ENCODE JSON;

In [ ]:
CREATE MATERIALIZED VIEW IF NOT EXISTS client_latest_mv AS
SELECT c.jedinstveni_identifikator, c.datum_registracije, c.starost, c.pol, c.bracno_stanje, MAX(p.datum) AS poslednja_posjeta
FROM client_source c
JOIN product_change_source p ON c.jedinstveni_identifikator = p.jedinstveni_identifikator
GROUP BY c.jedinstveni_identifikator, c.datum_registracije, c.starost, c.pol, c.bracno_stanje;

In [ ]:
CREATE MATERIALIZED VIEW product_window_summary AS
SELECT
    jedinstveni_identifikator,
    window_start,
    window_end,
    CASE WHEN SUM(CASE WHEN proizvod = 'pozajmica' AND 'indikator' = 'uzmi' THEN 1 ELSE -1 END) > 0 THEN 1 ELSE 0 END AS pozajmica,
    CASE WHEN SUM(CASE WHEN proizvod = 'kredit' AND 'indikator' = 'uzmi' THEN 1 ELSE -1 END) > 0 THEN 1 ELSE 0 END AS kredit,
    CASE WHEN SUM(CASE WHEN proizvod = 'elektronsko bankarstvo' AND 'indikator' = 'uzmi' THEN 1 ELSE -1 END) > 0 THEN 1 ELSE 0 END AS elektronsko_bankarstvo,
    CASE WHEN SUM(CASE WHEN proizvod = 'kreditna kartica' AND 'indikator' = 'uzmi' THEN 1 ELSE -1 END) > 0 THEN 1 ELSE 0 END AS kreditna_kartica             
FROM TUMBLE(product_change_source, datum, INTERVAL '1 HOUR') 
WHERE window_start = (
    SELECT MAX(window_start)
    FROM TUMBLE(product_change_source, datum, INTERVAL '1 HOUR')
)
GROUP BY
    jedinstveni_identifikator, window_start, window_end;  

In [ ]:
CREATE MATERIALIZED VIEW transaction_window_summary AS
SELECT
    jedinstveni_identifikator,
    COUNT(CASE WHEN vrsta_transakcije = 'isplate i uplate šalter' THEN 1 ELSE 0 END) AS isplate_i_uplate_salter_broj,
    SUM(CASE WHEN vrsta_transakcije = 'isplate i uplate šalter' THEN iznos_transakcije ELSE 0 END) AS isplate_i_uplate_salter_iznos,
    COUNT(CASE WHEN vrsta_transakcije = 'isplate i uplate bankomat' THEN 1 ELSE 0 END) AS isplate_i_uplate_bankomat_broj,
    SUM(CASE WHEN vrsta_transakcije = 'isplate i uplate bankomat' THEN iznos_transakcije ELSE 0 END) AS isplate_i_uplate_bankomat_iznos,
    COUNT(CASE WHEN vrsta_transakcije = 'nalozi šalter' THEN 1 ELSE 0 END) AS nalozi_salter_broj,
    SUM(CASE WHEN vrsta_transakcije = 'nalozi šalter' THEN iznos_transakcije ELSE 0 END) AS nalozi_salter_iznos,
    COUNT(CASE WHEN vrsta_transakcije = 'nalozi devizni šalter' THEN 1 ELSE 0 END) AS nalozi_devizni_salter_broj,
    SUM(CASE WHEN vrsta_transakcije = 'nalozi devizni šalter' THEN iznos_transakcije ELSE 0 END) AS nalozi_devizni_salter_iznos,
    COUNT(CASE WHEN vrsta_transakcije = 'nalozi e-bankarstvo' THEN 1 ELSE 0 END) AS nalozi_ebankarstvo_broj,
    SUM(CASE WHEN vrsta_transakcije = 'nalozi e-bankarstvo' THEN iznos_transakcije ELSE 0 END) AS nalozi_ebankarstvo_iznos,
    COUNT(CASE WHEN vrsta_transakcije = 'nalozi smart-atm' THEN 1 ELSE 0 END) AS nalozi_smartatm_broj,
    SUM(CASE WHEN vrsta_transakcije = 'nalozi smart-atm' THEN iznos_transakcije ELSE 0 END) AS nalozi_smartatm_iznos,
    COUNT(CASE WHEN vrsta_transakcije = 'trajni nalog' THEN 1 ELSE 0 END) AS trajni_nalog_broj,
    SUM(CASE WHEN vrsta_transakcije = 'trajni nalog' THEN iznos_transakcije ELSE 0 END) AS trajni_nalog_iznos,
    COUNT(CASE WHEN vrsta_transakcije = 'plaćanje i isplata kreditna kartica' THEN 1 ELSE 0 END) AS placanje_i_isplata_kreditna_kartica_broj,
    SUM(CASE WHEN vrsta_transakcije = 'plaćanje i isplata kreditna kartica' THEN iznos_transakcije ELSE 0 END) AS placanje_i_isplata_kreditna_kartica_iznos,
    COUNT(CASE WHEN vrsta_transakcije = 'plaćanje kreditna kartica web' THEN 1 ELSE 0 END) AS placanje_kreditna_kartica_web_broj,
    SUM(CASE WHEN vrsta_transakcije = 'plaćanje kreditna kartica web' THEN iznos_transakcije ELSE 0 END) AS placanje_kreditna_kartica_web_iznos
FROM TUMBLE(transaction_source, datum_transakcije, INTERVAL '1 HOUR')
WHERE window_start = (
    SELECT MAX(window_start)
    FROM TUMBLE(transaction_source, datum_transakcije, INTERVAL '1 HOUR')
)
GROUP BY
    jedinstveni_identifikator;

In [ ]:
CREATE MATERIALIZED VIEW agregirani_mv AS
SELECT
    c.jedinstveni_identifikator, 
    c.datum_registracije, 
    c.starost, 
    c.pol, 
    c.bracno_stanje, 
    c.poslednja_posjeta,
    p.pozajmica,
    p.kredit,
    p.elektronsko_bankarstvo,
    p.kreditna_kartica,
    t.isplate_i_uplate_salter_broj,
    t.isplate_i_uplate_salter_iznos,
    t.isplate_i_uplate_bankomat_broj,
    t.isplate_i_uplate_bankomat_iznos,
    t.nalozi_salter_broj,
    t.nalozi_salter_iznos,
    t.nalozi_devizni_salter_broj,
    t.nalozi_devizni_salter_iznos,
    t.nalozi_ebankarstvo_broj,
    t.nalozi_ebankarstvo_iznos,
    t.nalozi_smartatm_broj,
    t.nalozi_smartatm_iznos,
    t.trajni_nalog_broj,
    t.trajni_nalog_iznos,
    t.placanje_i_isplata_kreditna_kartica_broj,
    t.placanje_i_isplata_kreditna_kartica_iznos,
    t.placanje_kreditna_kartica_web_broj,
    t.placanje_kreditna_kartica_web_iznos
FROM
    client_latest_mv c
LEFT JOIN
    product_window_summary p ON c.jedinstveni_identifikator = p.jedinstveni_identifikator
LEFT JOIN
    transaction_window_summary t ON c.jedinstveni_identifikator = t.jedinstveni_identifikator;